In [1]:
import numpy as np
import torch

from torch.utils.data import Dataset

import torch.nn.functional as F
import torch.nn as nn

import matplotlib.pyplot as plt

import myModelinTF

from torch.optim.lr_scheduler import StepLR
import pandas as pd
import tensorflow as tf
from torch.autograd import Variable 

from statistics import mean 
import random

In [2]:
demo = np.load('dataset_train_firsts.npy', allow_pickle=True)
print("Train firsts set size: " + str(len(demo)))

Train firsts set size: 495


In [3]:
# def permute_them(x):
#   return torch.permute(x, (1, 0, 2))

In [4]:
def fill_four_with_last(arr):
    prev = np.arange(len(arr))
    prev[arr == 4] = 0
    prev = np.maximum.accumulate(prev)

    # it can be that the first element is 4, so remove those
    temp = arr[prev]
    new_deleted = np.delete(temp, np.where(temp == 4))
    count = len(temp) - len(new_deleted)
    new = np.insert(new_deleted, 0, np.repeat(new_deleted[0], count), axis=0)
    return new

In [5]:
class PG(nn.Module):
    def __init__(self, n_actions):
        super().__init__()
        self.n_actions = n_actions

        # Policy takes input initial conditions and outputs planning route, open-loop control
        self.actor_model = nn.Sequential(
            nn.LSTM(input_size = 1, hidden_size = self.n_actions, num_layers = 2, dropout = 0.1,batch_first = True),
        )

        self.logprobs = []
        self.state_values = []
        self.rewards = []

    def forward(self, states):
        logits, (h_T, c_T)  = self.actor_model(states)
        return logits[-1]
    # Run agent in environment to create sample trajectories by generator
    # The environment model is a seq2seq model
    def generate_session(self, sysmodel, init, traj_idx,  t_max=200):
        norm_params = pd.read_csv('D:\\units\\thesis\\thesis\\train_reward_inverse_rl_w_sys_model\\norm_params.csv', header = None).to_numpy().T
        norm_params_actions = pd.read_csv('D:\\units\\thesis\\thesis\\train_reward_inverse_rl_w_sys_model\\norm_params_actions.csv', header = None).to_numpy().T

        states, actions, actions_probs_policy, rewards = [], [], [], []
        init_idx = init[traj_idx]
        states_init_idx = init_idx[0]
        init_COT_idx = states_init_idx[1,0:20]

        states = torch.FloatTensor(init_COT_idx.T)

        states_history = states.unsqueeze(0)
        #states_history = states[-1,:].unsqueeze(0)
        list_actions_taken = []
        actions_probs_policy_taken = []
        action = 0
        for t in range(0, t_max, 20):
            logits = policy(states_history) # forward pass
            
            probs = nn.functional.softmax(logits, -1) # get estimated actions for states

            actions_probs_policy = np.squeeze(probs).clone().detach().numpy()
            actions_probs_policy_rep = np.tile(actions_probs_policy,(20,1))
            actions_probs_policy_taken.append(actions_probs_policy_rep)

            action_prev = action
            action = np.random.choice(3,  p = actions_probs_policy)
            if action == 2:
                action = action_prev

            action_rep = np.repeat(action, 20)
            list_actions_taken.append(action_rep)
            
            #list_actions_filled = np.append(list_actions_filled, a)
            
            # Apply dict to go from 1,2,3,4 as action to [0,2400,0] [0,2400,1000]  [2300,0,0]  [2300,0,1000] 
            my_dict = {0:[0,2400,1000], 1:[2300,0,1000], 2: [2300,0,0]} 
            list_actions_taken_np = np.array(list_actions_taken) 
            list_actions_taken_np = np.append([2, 2, 2, 2, 2], list_actions_taken_np)
            actions_array_mapped = map(my_dict.get, list_actions_taken_np.flatten())
            actions_array_mapped = np.array(list(actions_array_mapped))
            
            actions_array_mapped = (actions_array_mapped - norm_params_actions[:,0]) / norm_params_actions[:,1]

            actions_sysmodel = np.concatenate([np.arange(actions_array_mapped.shape[0])[:,None]+1,actions_array_mapped], axis=1) #maybe needed [:,None]
            actions_sysmodel = tf.expand_dims(actions_sysmodel, axis = 0, name=None)
            
            states = sysmodel(actions_sysmodel)

            # Revert normalized values for states

            states = states*norm_params[:,1]+norm_params[:,0]
            states_COT_np = np.array(states[:,:,0])
            states = torch.FloatTensor(states_COT_np[:,:-1])
            # More actions fed to the sysmodel, higher the accuracy, but the resulting states will not be the ones effectively experienced, so history is needed
            # Rewards can be calculated on the set of states experienced with all actions at once, but then the scope is just to evaluate the set of actions in the whole episode, not step by step
            states_history = torch.cat((states_history, states[-20:]), 1)

            # if done:
            #     break
        
        # print(actions_taken)
        # actions_taken = np.array(actions_taken)
        actions_probs_policy_taken = np.array(actions_probs_policy_taken).reshape(t_max, 3)
        # states_all = sysmodel(actions_taken_reshape)
        # states_all = states_new*norm_params[:,0]+norm_params[:,1]
        rewards = get_rewards(states, np.array(list_actions_taken), t_max)
        return states_history, states, list_actions_taken_np.flatten(), actions_probs_policy_taken, rewards

In [6]:
class ValueFunc(nn.Module):
    def __init__(self):
        super().__init__()

        self.value_model = nn.Sequential(
            nn.Linear(5, 32),
            nn.ReLU(inplace=False),
            nn.Linear(32, 1)
        )

    def forward(self, state):
        state = torch.FloatTensor(state) #.unsqueeze(0)
        state_value = self.value_model(state)
        return state_value

In [7]:
def ind(array, t_max):
    idx_return = (t_max + 1,0)
    for idx, val in np.ndenumerate(array):
        if val <= 201 and val >= 199:
            idx_return = idx
            break
    return int(idx_return[0])
    # If no item was found return None, other return types might be a problem due to
    # numbas type inference.

In [11]:
def get_y_rewards(model,states, rewards_loc,gamma=0.9):
    states = states.numpy()
    for idx_i in range(len(rewards_loc)):
        for idx in range(idx_i + 1, len(rewards_loc)):
            input1 = states[:,idx]
            for i in range(4):
                input1 = np.concatenate((input1,[np.max(states[:,idx-5*(i+1):])]))
            out1 = model(input1).squeeze()
            rewards_loc[idx_i] += out1.detach().numpy()
            continue
    return rewards_loc

def get_expected_rewards(model,states, rewards,gamma=0.9):
    G = np.zeros_like(rewards, dtype=float)
    states = states.numpy()
    for idx in range(-1, -len(rewards), -1):
        input1 = states[:,idx]
        for i in range(4):
            input1 = np.concatenate((input1, [np.max(states[:,idx-5*(i+1):])]))
        G[idx] = model(input1).detach().numpy() + G[idx+1]
    return G

# For REINFORCE with baseline
def get_rewards_to_go(rewards,gamma=0.9):
    G = np.zeros_like(rewards, dtype=float)
    for idx in range(-1, -len(rewards), -1):
        G[idx] = rewards[idx] + gamma * G[idx+1]

    return G

def get_advantage_rewards(model,states, rewards,gamma=0.9):
    G = np.zeros_like(rewards, dtype=float)
    #G[-1] = rewards[-1]
    states = states.numpy()
    for idx in range(-1, -len(rewards)-1, -1):
        input1 = states[:,idx]
        for i in range(4):
            input1 = np.concatenate((input1,[np.mean(states[:,idx-5*(i+1):])]))

        input2 = states[:,idx + 1]
        for i in range(4):
            input2 = np.concatenate((input2, [np.mean(states[:,(idx + 1)-5*(i+1):])]))

        out1 = model(input1) + out1
        out2 = model(input2)
        G[idx] = rewards[idx] + out2.detach().numpy() - out1.detach().numpy()
    return G

def get_rewards(states, actions, t_max):
    
    states = states.numpy()
    states = states.squeeze()
    rewards = np.zeros((np.shape(states)[0],1))

    actions = actions.flatten()
    # print(actions)
    max_hotspot =  np.max(states)
    if max_hotspot < 280:
        rewards[-1] = 1

    target_first_reached_at = ind(states,t_max)

    print(f"Reached Target 230C at step {target_first_reached_at} with t_max = {t_max}")

    if target_first_reached_at < t_max and target_first_reached_at > (t_max - 180):
        rewards[-1] = rewards[-1] + 15

        
    for idx in range(np.shape(states)[0]):
        #idx = idx -1
        # if temps.any() > 230:
        #     rewards[idx] = rewards[idx] -30
        
        # subset = actions[idx-20:]
        # if np.sum(np.diff(subset)) == 0:
        #     rewards[idx] = rewards[idx] + 20

        # subset = states[idx-10:]
        # if np.absolute(np.sum(np.diff(subset))) < 10:
        #     #print('Difference between states not big')
        #     rewards[idx] = rewards[idx] + 70

        # Reduce power consumption
        # if actions[idx] == 1 | actions[idx] == 3:
        #     rewards[idx] = rewards[idx] -10

        # Action 4 is encouraged , has the meaning to keep the previous action
        # if actions[idx] == 2:
        #     #print('Reward for keeping previous action')
        #     rewards[idx] = rewards[idx] + 1

        # Negative temp goes with high negative reward
        if states[idx] < 0:
            #print('Penalizing for negative values')
            rewards[idx] = rewards[idx] -2

        # Penalize if previous state higher than current (variation)
        temps_subset = np.array([states[idx], states[idx-1]])
        if np.diff(temps_subset) <= 0:
            #print('Penalizing for decreasing or steady state value')
            rewards[idx] = rewards[idx] - 5

        if 0 > np.diff(temps_subset) <= 1 :
            #print('Penalizing for decreasing or steady state value')
            rewards[idx] = rewards[idx] + 2

    return rewards

def to_one_hot(y_tensor, ndims):
    y_tensor = y_tensor.type(torch.LongTensor).view(-1, 1)
    y_one_hot = torch.zeros(
        y_tensor.size()[0], ndims).scatter_(1, y_tensor, 1)
    return y_one_hot

In [9]:
def sample_demos(demos, traj_idx):
    demo = demos[traj_idx]
    states_demo = demo[0]
    
    COT_demo = states_demo[1]
    states = torch.FloatTensor(COT_demo.T)
    states = states.unsqueeze(0)

    list_actions_taken = np.array(demo[1])
    actions_probs_policy_taken = demo[2]

    rewards = get_rewards(states, list_actions_taken, 480)
    
    return states, list_actions_taken, actions_probs_policy_taken, rewards

In [12]:
# DEVICE
DEVICE = 'cpu'

# ENV SETUP
sysmodel = myModelinTF.load_model()
n_actions = 3 # excluded 1:[0,2400,1000], 3:[2300,0,1000], then returned now excluded {0:[0,2400,0], 1:[2300,0,0]}

# INITILIZING POLICY AND REWARD FUNCTION
policy = PG(n_actions)
policy.to(DEVICE)

value = ValueFunc()
value.to(DEVICE)

norm_params = pd.read_csv('D:\\units\\thesis\\thesis\\train_reward_inverse_rl_w_sys_model\\norm_params.csv', header = None).to_numpy().T
norm_params_actions = pd.read_csv('D:\\units\\thesis\\thesis\\train_reward_inverse_rl_w_sys_model\\norm_params_actions.csv', header = None).to_numpy().T


# LOADING EXPERT/DEMO SAMPLES
DATASETNPY = "dataset_train_firsts.npy"

DATASET_DEMO = "dataset_train.npy"

##### TUNE MODEL WITH REWARD LOSS

mean_rewards = []
mean_loss = []
mean_loss_rew = []

# EACH BATCH HAS TRAJECTORIES
EPISODES_FROM_DEMO = 10
EPISODES_TO_PLAY = 5

BATCHES_DEMO = 100
ITERS_POLICY = 100

sample_trajs = []

return_list, sum_of_loss_list, loss_value_list = [], [], []

prev_R_lst = []

# Retrain all params
optimizer_policy = torch.optim.Adam(policy.parameters(), 1e-1, weight_decay=1e-2)
optimizer_value = torch.optim.Adam(value.parameters(), 1e-1, weight_decay=1e-2)

init_states = np.load(DATASETNPY, allow_pickle=True)
demos = np.load(DATASET_DEMO, allow_pickle=True)

criterion2 = nn.MSELoss()

t_max = 500

#Epochs Demo

# Episodes
for i in range(BATCHES_DEMO):
    trajs_demo = [sample_demos(demos, l) for l in random.sample(range(400), EPISODES_FROM_DEMO)] 
    expected_rewards_lst = []
    rewards_to_go_lst = []
    for traj in trajs_demo:
        states, actions, probs, rewards = traj
        
        rewards_plot = np.array(rewards.squeeze())
        expected_returns_np = np.array(get_expected_rewards(value,states,rewards, 0.7)).squeeze()
        expected_returns = torch.tensor(expected_returns_np, dtype=torch.float32)

        # R_np = np.array(get_rewards_to_go(rewards, 0.7)).squeeze()
        # R = torch.tensor(R_np, dtype=torch.float32)

        # #cumulative_returns = (R - expected_returns) 
        # cumulative_returns = (R - torch.mean(torch.from_numpy(np.array(prev_rewards).flatten())))
        # #baseline with mean rewards of sampled trajectories

        # actions_tensor = torch.from_numpy(actions)
        # probs_tensor = torch.from_numpy(np.array(probs))

        # log_probs_for_actions = -torch.sum(to_one_hot(actions_tensor, 4), dim =1) - 0.2

        # loss_policy = -log_probs_for_actions*cumulative_returns/1e6
        # # UPDATING THE POLICY NETWORK
        # optimizer_policy.zero_grad()
        # loss_policy.requires_grad = True
        # loss_policy.sum().backward()
        # torch.nn.utils.clip_grad_norm_(policy.parameters(), 0.01)
        # optimizer_policy.step()
        expected_rewards_lst.append(get_expected_rewards(value,states,rewards, 0.7).sum())
        rewards_to_go_lst.append(get_rewards_to_go(rewards, 0.7).sum())

    loss_value = criterion2(torch.from_numpy(np.array(mean(expected_rewards_lst))).to(torch.float32), \
                                torch.from_numpy(np.array(mean(rewards_to_go_lst))).to(torch.float32))/1e10
    # UPDATING THE VALUE NETWORK
    optimizer_value.zero_grad()
    loss_value.requires_grad = True
    loss_value.backward()
    optimizer_value.step()

    loss_value_list.append(loss_value.detach().numpy())

    #returns = sum(rewards)
    #sum_of_loss = loss_policy.sum().detach().numpy()
    #print(f"\nLoss Actor: {sum_of_loss}\n")
    print(f"\nLoss Value: {loss_value}\n")

    #sum_of_loss_list.append(sum_of_loss)
    states_plot = np.array(states.squeeze())

    # plt.subplot(2, 3, 1)
    # plt.title(f"Policy Loss")
    # plt.plot(sum_of_loss_list)
    # plt.grid()

    plt.subplot(2, 3, 2)
    plt.title(f"Value Loss")
    plt.plot(loss_value_list)
    plt.grid()

    plt.subplot(2, 3, 3)
    plt.title(f"Rewards")
    plt.plot(rewards_plot)
    plt.grid()

    plt.subplot(2, 3, 4)
    plt.title(f"Activations")
    plt.plot(actions[0:500])
    plt.grid()

    plt.subplot(2, 3, 6)
    plt.title(f"Temps Planned")
    plt.plot(states_plot[0:500], color = 'm')
    plt.grid()

    plt.savefig('GCL_learning_curve.png')
    plt.close()

rewards_sum_plot = np.array((0))
log_probs_for_actions_lst, cumulative_returns_lst = [], []
for i in range(ITERS_POLICY):
    trajs_policy = [policy.generate_session(sysmodel, init_states, l, t_max) for l in random.sample(range(400), EPISODES_TO_PLAY)]

    log_probs_for_actions_lst, cumulative_returns_lst, expected_rewards_lst, rewards_to_go_lst = [], [], [], []

    for traj in trajs_policy:
        states_history, states, actions, probs, rewards = traj

        rewards_sum_plot = np.column_stack((rewards_sum_plot, np.array(rewards.sum())))

        expected_returns_np = np.array(get_expected_rewards(value,states,rewards, 0.7)).squeeze()
        expected_returns = torch.tensor(expected_returns_np, dtype=torch.float32)

        R_np = np.array(get_rewards_to_go(rewards, 0.7)).squeeze()
        R = torch.tensor(R_np, dtype=torch.float32)

        prev_R_lst.append(R_np)

        cumulative_returns = (R - expected_returns) 
        #cumulative_returns = (R - torch.mean(torch.from_numpy(np.array(prev_R_lst).flatten())))

        actions_tensor = torch.from_numpy(actions)
        probs_tensor = torch.from_numpy(np.array(probs))

        log_probs_for_actions = torch.log(torch.sum(
            probs_tensor * to_one_hot(actions_tensor[5:], 3), dim =1))
        
        log_probs_for_actions_lst.append(log_probs_for_actions)
        cumulative_returns_lst.append(cumulative_returns)

        expected_rewards_lst.append(get_expected_rewards(value,states,rewards, 0.7).sum())
        rewards_to_go_lst.append(get_rewards_to_go(rewards, 0.7).sum())
        

    loss_policy = -torch.mean(torch.from_numpy(np.array(log_probs_for_actions_lst)))*torch.mean(torch.from_numpy(np.array(cumulative_returns_lst)))
    # UPDATING THE POLICY NETWORK
    optimizer_policy.zero_grad()
    loss_policy.requires_grad = True
    loss_policy.sum().backward()
    torch.nn.utils.clip_grad_norm_(policy.parameters(), 0.01)
    optimizer_policy.step()

    loss_value = criterion2(torch.from_numpy(np.array(mean(expected_rewards_lst))).to(torch.float32), \
                                torch.from_numpy(np.array(mean(rewards_to_go_lst))).to(torch.float32)) /1e10
    # UPDATING THE VALUE NETWORK
    optimizer_value.zero_grad()
    loss_value.requires_grad = True
    loss_value.backward()
    torch.nn.utils.clip_grad_norm_(value.parameters(), 0.01)
    optimizer_value.step()

    #returns = sum(rewards)
    sum_of_loss = loss_policy.sum().detach().numpy()
    print(f"\nLoss Actor: {sum_of_loss}\n")
    print(f"\nLoss Value: {loss_value}\n")

    sum_of_loss_list.append(sum_of_loss)
    states_history_plot = np.array(states_history.squeeze())
    states_plot = np.array(states.squeeze())
    loss_value_list.append(loss_value.detach().numpy())
    rewards_sum_plot2 = rewards_sum_plot.squeeze()

    plt.subplot(2, 3, 1)
    plt.title(f"Policy Loss")
    plt.plot(sum_of_loss_list)
    plt.grid()

    plt.subplot(2, 3, 2)
    plt.title(f"Value Loss")
    plt.plot(loss_value_list)
    plt.grid()

    plt.subplot(2, 3, 3)
    plt.title(f"Rewards Sum")
    plt.plot(rewards_sum_plot2[1:])
    plt.grid()

    plt.subplot(2, 3, 4)
    plt.title(f"Activations")
    plt.plot(actions[0:500])
    plt.grid()

    plt.subplot(2, 3, 5)
    plt.title(f"Temps Actual")
    plt.plot(states_history_plot[0:500].squeeze(), color='red')
    plt.grid()

    plt.subplot(2, 3, 6)
    plt.title(f"Temps Planned")
    plt.plot(states_plot[0:500].squeeze(), color = 'm')
    plt.grid()

    plt.savefig('GCL_learning_curve.png')
    plt.close()

Reached Target 230C at step 421 with t_max = 480
Reached Target 230C at step 421 with t_max = 480
Reached Target 230C at step 419 with t_max = 480
Reached Target 230C at step 415 with t_max = 480
Reached Target 230C at step 416 with t_max = 480
Reached Target 230C at step 418 with t_max = 480
Reached Target 230C at step 419 with t_max = 480
Reached Target 230C at step 417 with t_max = 480
Reached Target 230C at step 418 with t_max = 480
Reached Target 230C at step 417 with t_max = 480

Loss Value: 56.03561782836914

Reached Target 230C at step 415 with t_max = 480
Reached Target 230C at step 425 with t_max = 480
Reached Target 230C at step 418 with t_max = 480
Reached Target 230C at step 417 with t_max = 480
Reached Target 230C at step 420 with t_max = 480
Reached Target 230C at step 433 with t_max = 480
Reached Target 230C at step 417 with t_max = 480
Reached Target 230C at step 439 with t_max = 480
Reached Target 230C at step 428 with t_max = 480
Reached Target 230C at step 418 with 

: 